In [11]:
# multivariate output 1d cnn example
from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from numpy import loadtxt
lines = loadtxt("trial0_revised11.txt", delimiter=",", unpack=False, usecols=(4,7,10,13,16,19,22,25,28,31))
print(lines[0][0])


0.160804


In [12]:
len(lines)

24003

In [13]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	print(len(sequences))
	for i in range(len(sequences)):
		seq_x, seq_y = sequences[i, :7], sequences[i, 7:]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


In [14]:

# horizontally stack columns
#dataset = hstack((in_seq1, in_seq2, out_seq))
#print(dataset)
# choose a number of time steps
n_steps = 3


In [15]:
# convert into input/output
X, y = split_sequences(lines, n_steps)
print(X.shape)
print(y.shape)

24003
(24003, 7)
(24003, 3)


In [16]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[-1]
print('shape='+str(n_features))

shape=7


In [21]:
visible = Input(shape=(3, n_features))
cnn = Conv1D(filters=64, kernel_size=2, activation='relu')(visible)
cnn = MaxPooling1D(pool_size=2)(cnn)
cnn = Flatten()(cnn)
cnn = Dense(50, activation='relu')(cnn)
# define output 1
output1 = Dense(1)(cnn)
# define output 2
output2 = Dense(1)(cnn)
# define output 3
output3 = Dense(1)(cnn)


In [23]:
# tie together
model = Model(inputs=visible, outputs=[output1, output2, output3])
model.compile(optimizer='adam', loss='mse')

y1 = y[:, 0].reshape((y.shape[0], 1))
y2 = y[:, 1].reshape((y.shape[0], 1))
y3 = y[:, 2].reshape((y.shape[0], 1))


# fit model
print('fitting')
model.fit(X, [y1,y2,y3], epochs=2000, verbose=0)

fitting


ValueError: in user code:

    File "c:\Users\shoba\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\shoba\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\shoba\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\shoba\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\shoba\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\shoba\App